# Error Visualization

- Quantifying Errors
	- From Determined Test Cases,  
For each word, measure the number of *correctly* and *incorrectly* interpreted
- Classifying Errors
	- Miss interpretation of the words  
For each word, log the transcription, what is the the misinterpretation of the error  
→ calculate the occurence  
→ most common error for a given word


In [1]:
import os
import numpy
import glob
import jiwer
import copy

def idx_to_file(idx):
    return "/".join(idx.split("-")[:-1])


def read_transcription(fpath):
    file = open(fpath)
    transcription = file.readline()
    file.close()

    return transcription


In [9]:
import jiwer

def preprocess_text(text):
    text = text.lower()
    text = jiwer.RemoveMultipleSpaces()(text)
    text = jiwer.ExpandCommonEnglishContractions()(text)
    text = jiwer.RemoveWhiteSpace(replace_by_space=True)(text) # must remove trailing space after it
    text = jiwer.Strip()(text)
    return text


In [10]:
from asr_evaluation.asr_evaluation import asr_evaluation

In [19]:
root_dir = "LibriSpeech/test-clean/"

model_dir = "deepspeech"
# model_dir = "finetuned_deepspeech"

wers = []
infos = []

### root_dir needs a trailing slash (i.e. /root/dir/)
for filename in glob.iglob(root_dir + '**/*.trans.txt', recursive=True):
    # print(filename)

    # filename = "LibriSpeech/test-clean/61/70968/61-70968.trans.txt"

    
    file = open(filename)

    for line in file.readlines():
    # for line in file.readlines()[0:5]:
        idx = line.split()[0]
        reference_text = " ".join(line.split()[1:])

        fname = os.path.join(root_dir, idx_to_file(idx), idx)
        flac_path = fname + ".flac"
        wav_path = fname + ".wav"
        transcription_path = fname + "." + model_dir + ".transcription.txt"

        if os.path.exists(transcription_path):
            transcription = read_transcription(transcription_path)

            transcription = preprocess_text(transcription)
            reference_text = preprocess_text(reference_text)
            
            wer = jiwer.wer(reference_text, transcription)
            wers.append(wer)
            if wer != 0 :
                evaluation = asr_evaluation.ASREvaluation()
                evaluation.detect_word_error(reference_text, transcription)
                confusion = evaluation.get_confusions()
                infos.append({"confusion" : confusion, "reference_text": reference_text, "transcription": transcription})
        else :
            # continue
            raise ValueError("missing transcription: " + transcription_path)
        
    file.close()

print(f"Average WER: {100 * sum(wers) / len(wers):.2f}%")


Average WER: 8.26%


In [20]:
def print_info(info):
    print("Reference: \t", info["reference_text"])
    print("Transcription: \t", info["transcription"])
    print("Confusion:")
    confusion = info["confusion"]
    if len(confusion["substitution"]) > 0:
        print("\tSubstitution:")
        for i in range(len(confusion["substitution"])):
            print(f"\t\t{confusion['substitution'][i]['word_reference']:10s} -> {confusion['substitution'][i]['word_substitution']:10s} \t count: {confusion['substitution'][i]['count']}")
    if len(confusion["insertion"]) > 0:
        print("\tInsertion:")
        for i in range(len(confusion["insertion"])):
            print(f"\t\t{confusion['insertion'][i]['word']:10s} \t count: {confusion['insertion'][i]['count']}")
    if len(confusion["deletion"]) > 0:
        print("\tDeletion:")
        for i in range(len(confusion["deletion"])):
            print(
                f"\t\t{confusion['deletion'][i]['word']:10s} \t count: {confusion['deletion'][i]['count']}")


In [21]:
print_info(infos[0])

Reference: 	 he knew the silver fleece his and zora is must be ruined
Transcription: 	 he knew the silver fleece his enormous be ruined
Confusion:
	Substitution:
		must       -> enormous   	 count: 1
	Deletion:
		and        	 count: 1
		zora       	 count: 1
		is         	 count: 1


In [22]:
print_info(infos[1])


Reference: 	 he panted to know if she too knew or knew and cared not or cared and knew not
Transcription: 	 he wanted to know if she too knew or knew and cared not or cared and knew not
Confusion:
	Substitution:
		panted     -> wanted     	 count: 1


In [23]:
print_info(infos[5])


Reference: 	 the squares of cotton sharp edged heavy were just about to burst to bolls
Transcription: 	 the squares of cotton sharp edge heavy were just about to burst to bulls
Confusion:
	Substitution:
		edged      -> edge       	 count: 1
		bolls      -> bulls      	 count: 1


In [24]:
substitution_errors = {}

for info in infos:
    confusion = info["confusion"]
    if len(confusion["substitution"]) > 0:
        for i in range(len(confusion["substitution"])):
            word_reference = confusion["substitution"][i]["word_reference"]
            word_substitution = confusion["substitution"][i]["word_substitution"]
            count = confusion["substitution"][i]["count"]
            if word_reference in substitution_errors:
                substitutions = substitution_errors[word_reference] 
                if word_substitution in substitutions :
                    substitution_errors[word_reference][word_substitution] = count + substitution_errors[word_reference][word_substitution]
                else :
                    substitution_errors[word_reference][word_substitution] = count
            else :
                substitution_errors[word_reference] = {word_substitution: count}

substitution_errors

{'must': {'enormous': 1, 'i': 1},
 'panted': {'wanted': 1},
 'at': {'admit': 1,
  'its': 1,
  'above': 1,
  'and': 4,
  'had': 1,
  'of': 2,
  'noted': 1,
  'in': 1,
  'a': 1,
  'that': 2,
  'stared': 1,
  'it': 1},
 'midday': {'day': 1},
 'and': {'a': 3,
  'hand': 1,
  'in': 28,
  'when': 1,
  'of': 1,
  'torn': 1,
  'worth': 1,
  'but': 1,
  'coming': 1,
  'i': 1,
  'noting': 1,
  'women': 1,
  'wind': 1,
  'ranking': 1,
  'on': 2,
  'loosen': 1,
  'government': 1,
  'at': 1,
  'phranza': 1,
  'cousin': 1,
  'as': 2,
  'nation': 1,
  'join': 1,
  'that': 1,
  'to': 1,
  'rockin': 1,
  'maiden': 1,
  'then': 1,
  'leaped': 1,
  'read': 1,
  'massan': 1,
  'wherein': 1},
 'the': {'a': 7,
  'me': 2,
  'cared': 1,
  'for': 1,
  'tender': 1,
  'common': 1,
  'that': 1,
  'new': 1,
  'but': 2,
  'death': 1,
  'any': 1,
  'were': 1,
  'consate': 1,
  'at': 1,
  'their': 1,
  'guilty': 1,
  'to': 2,
  'gift': 1,
  'apolline': 1,
  'turn': 1},
 'edged': {'edge': 1},
 'bolls': {'bulls': 1},
 '